# Using templates

dif do ponto mais acima e a esquerda do template.

ponto inicial, esquerda de cima (-20 px, -90 px)

ponto final, direita de baixo (2038px, 128px)


1530 x 459
2848 x 2517

carta tem (1318 x 2058)

In [144]:
## Base code generated by ChatGPT

import cv2
import numpy as np
from imutils.object_detection import non_max_suppression

# Load the input image
img = cv2.imread("./cartas/digitalizar0001.jpg")

# Convert the input image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Load the pattern image
template = cv2.imread("./template/template_medido_topo_para_esquerda.jpg", cv2.IMREAD_GRAYSCALE)

w, h = template.shape[::-1]

# Perform template matching to find the pattern in the image
result = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)

# Set a threshold for the match
threshold = 0.751 #value hand tunned

# Get all matches that exceed the threshold
locations = np.where(result >= threshold) # np.where catches indices of all elements that are greater than the threshold

#Save all matches in a box list
boxes = []
for pt in zip(*locations[::-1]):
    #(x, y, x+w, y+h)
    boxes.append((pt[0]-20, pt[1]-90, pt[0]+2038, pt[1]+1228))

#Apply non_max_supression on boxes
boxes = non_max_suppression(np.array(boxes))

# Save cards and save a final image showing saved places
i = 0
for box in boxes:
    # cv2.circle(img, (box[0], box[1]), 50, (255, 0, 0), 5)
    # cv2.circle(img, (box[0], box[1]), 10, (255, 0, 0), 5)
    cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 2)
    match = img[box[1]:box[3], box[0]:box[2]]
    
    #Save card
    # cv2.imwrite("./template_output/match_{}.jpg".format(i), match)
    
    i += 1

cv2.imwrite('template_extraction.jpg', img)

print(i)

12


# Using border detection

In [387]:
# Extract the inner part of Munchkin cards using their contour.

# alpha = brightness modifier
# beta = contrast modifier
# blur_kernel_size = how much the image is blurred before threshold applies
# threshold_value = color in which values above will become white.
# canny_threshold_1 = Value to mark image edges
# canny_threshold_2 = Value to mark image edges
# epsilon_multiplier = approximation ratio for cards contour
# aspect_ratio_tolerance = tolarence for card validation in regards to real cards aspect ratio

image = cv2.imread("./cartas/digitalizar0001.jpg")

# Apply brightness increase
alpha = 1.5 
beta = -0.5
dst = cv2.addWeighted(image, alpha, np.zeros(image.shape, image.dtype), 1 - alpha, beta)

# Convert image to grayscale
gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)

blur_kernel_size = (45, 45)
blur = cv2.GaussianBlur(gray, blur_kernel_size , 0)

# Threshold the image to keep only white pixels
threshold_value = 105
_, thresholded = cv2.threshold(blur, threshold_value, 255, cv2.THRESH_BINARY)

# Find edges
canny_threshold_1 = 110
canny_threshold_2 = 250
edges = cv2.Canny(thresholded, canny_threshold_1, canny_threshold_2)

# Find contours
contours, hierarchy = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

cards = []
aspect_ratio = 87/56
for contour in contours:
    # Approximate the contour
    epsilon_multiplier = 0.01
    epsilon = epsilon_multiplier * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    if len(approx) == 4:
        # Get the bounding rectangle
        x, y, w, h = cv2.boundingRect(approx)

        # Check the aspect ratio
        card_aspect_ratio = w / h
        aspect_ratio_tolerance = 0.1
        if abs(card_aspect_ratio - aspect_ratio) < aspect_ratio_tolerance:
            cards.append(contour)


In [394]:
# Draw the cards
#Reduce the archive to fit nicely the card, reduce repetitive cards
for i, card in enumerate(cards):
    mask = np.zeros(image.shape, np.uint8)
    cv2.drawContours(mask, [card], 0, (255, 255, 255), -1)
    region = cv2.bitwise_and(image, mask)

    # gray = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
    # _, contours = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # # Get the minimum bounding rectangle around the contour
    # x, y, w, h = cv2.boundingRect(contours)

    # # Extract the ROI
    # roi = region[y:y+h, x:x+w]
    
    cv2.imwrite("./contour_output/extracted_region_{}.jpg".format(i), region)
